# Pokemon Infinite Fusion Optimizer

Program created to simplify the process of finding the best fusion for a role, it lets you weight the stats, filter by name, total base stat and adjust the importance of the typing, both offensively and defensively.

You can also give more importance to specific types, rewarding crucial resistances or STABS such as fighting or ground in comparison to others such as bug. :)

Abilities and movesets are currently ignored, the latter can be added in the future but it probably won't.

In [64]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
from itertools import combinations

# PARAMETERS

# Weightings of HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, then offensive and defensive typing scores
# The weight the offensive and defensive typing scores receive directly affect the importance of the typing and the stats, consider them as percentages.
profiles = {
    'Fast physical attacker': ([1, 10, 1, 1, 1, 10], 0.3, 0.01),
    'Fast special attacker': ([1, 1, 1, 10, 1, 10], 0.3, 0.01),
    'Physical tank': ([8, 2, 8, 1, 4, 1], 0.01, 0.25),
    'Special tank': ([8, 1, 4, 2, 8, 1], 0.01, 0.25),
    'Hybrid tank': ([7, 1, 6, 2, 7, 1], 0.01, 0.3),
    'Balanced overall': ([3, 5, 3, 6, 3, 4], 0.2, 0.15),
    "Physical wallbreaker": ([6, 9, 4, 1, 3, 1], 0.2, 0.15),
    "Special wallbreaker": ([6, 1, 3, 9, 4, 1], 0.15, 0.2)
}

stat_cap = 570
use_filter = True

# Filtered pokemon list
unwanteds = ['Deoxys', 'Blissey', 'Shuckle', 'Chansey', 'Latios', 'Latias', 'Darkrai', 'Wobbuffet']

# Boosted types
boosted_types = ['fire', 'fighting', 'ground', 'water', 'dark']
boost = 1.5
# Nerfed types
nerfed_types = ['rock', 'bug', 'steel', 'grass']
nerf = 0.75

# Exceptions that have their secondary type removed
exceptions = ['Gyarados', 'Steelix', 'Dragonite', 'Moltres', 'Zapdos', 'Articuno', 'Scyther', 'Gengar', 'Haunter', 'Gastly', 'Onix', 'Golem', 'Graveler', 'Geodude', 'Charizard', 'Venusaur', 'Ivysaur', 'Bulbasaur']

Dataset extraction

In [65]:
def extract_table(url, class_name):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': class_name})

    data = []
    for row in table.find_all('tr'):
        row_data = []
        for cell in row.find_all('td'):
            row_data.append(cell.text.strip())
        data.append(row_data)
    return  pd.DataFrame([row for row in data if row != []])


infinite_mons = extract_table("https://infinitefusion.fandom.com/wiki/Pok%C3%A9dex", 'article-table')
infinite_mons = infinite_mons.iloc[:, :2]
infinite_mons.columns = ['Index', 'Name']

pokemon_set = extract_table("https://pokemon-index.com/base", 'move-lev by-base')
pokemon_set = pokemon_set.drop(columns=[0])
pokemon_set['Index'] = pokemon_set.index

Preprocessing

In [66]:
type_dict = {'nor': 'normal', 'fig': 'fighting', 'fly': 'flying', 'poi': 'poison', 'gro': 'ground', 'roc': 'rock', 'bug': 'bug', 'gho': 'ghost', 'ste': 'steel', 'fir': 'fire', 'wat': 'water', 'gra': 'grass', 'ele': 'electric', 'psy': 'psychic', 'ice': 'ice', 'dra': 'dragon', 'dar': 'dark', 'fai': 'fairy'}

pokemon_set['Type 1'] = ''; pokemon_set['Type 2'] = ''
pokemon_set['Type 1'] = pokemon_set[2].str.split(' ').str[0].map(type_dict)
pokemon_set['Type 2'] = pokemon_set[2].str.split(' ').str[1].map(type_dict)

pokemon_set = pokemon_set.drop(columns=[2])
pokemon_set.columns = ['Name', 'HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed', 'Total', 'Index', 'Type 1', 'Type 2']

Fusion of datasets and more preprocessing

In [67]:
pokemon_set = pd.merge(infinite_mons, pokemon_set, on='Name', how='right')
pokemon_set = pokemon_set.loc[((pokemon_set['Index_y'] < 252) & (pokemon_set['Index_y'].notnull())) | ((pokemon_set['Index_x'].notnull()) & (pokemon_set['Index_y'].notnull()))]

pokemon_set.drop(columns=['Index_x'], inplace=True)
pokemon_set.rename(columns={'Index_y': 'Index'}, inplace=True)
for col in ['HP', 'Attack', 'Defense', 'Special Attack', 'Special Defense', 'Speed', 'Total']: pokemon_set[col] = pokemon_set[col].astype(int)

In [68]:
#exceptions
pokemon_set.loc[pokemon_set['Name'].isin(exceptions), 'Type 2'] = np.nan

pokemon_set.loc[(pokemon_set['Type 1'] == 'normal') & (pokemon_set['Type 2'] == 'flying'), 'Type 1'] = 'flying'
pokemon_set.loc[(pokemon_set['Type 1'] == 'normal') & (pokemon_set['Type 2'] == 'flying'), 'Type 2'] = np.nan
pokemon_set.loc[pokemon_set['Name'] == 'Fletchling', 'Type 1'] = 'normal'
pokemon_set.loc[pokemon_set['Name'] == 'Fletchling', 'Type 2'] = 'flying'

names = ['Dewgong', 'Magnemite', 'Magneton', 'Magnezone', 'Omanyte', 'Omastar', 'Kabuto', 'Kabutops', 'Scizor', 'Empoleon', 'Spiritomb', 'Ferrothorn']
pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 1'], pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 2'] = pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 2'], pokemon_set.loc[pokemon_set['Name'].isin(names), 'Type 1']

if use_filter: pokemon_set = pokemon_set.loc[~pokemon_set['Name'].isin(unwanteds)]

Type related stuff

In [69]:
type_chart = {'normal': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': -2, 'bug': 1, 'ghost': -4, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'fighting': {'normal': 2, 'fighting': 1, 'flying': -2, 'poison': -2, 'ground': 1, 'rock': 2, 'bug': -2, 'ghost': -4, 'steel': 2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': -2, 'ice': 2, 'dragon': 1, 'dark': 2, 'fairy': -2},
            'flying': {'normal': 1, 'fighting': 2, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': -2, 'bug': 2, 'ghost': 1, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 2, 'electric': -2, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'poison': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': -2, 'ground': -2, 'rock': -2, 'bug': 1, 'ghost': -2, 'steel': -4, 'fire': 1, 'water': 1, 'grass': 2, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 2},
            'ground': {'normal': 1, 'fighting': 1, 'flying': -4, 'poison': 2, 'ground': 1, 'rock': 2, 'bug': -2, 'ghost': 1, 'steel': 2, 'fire': 2, 'water': 1, 'grass': -2, 'electric': 2, 'psychic': 1, 'ice': 1, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'rock': {'normal': 1, 'fighting': -2, 'flying': 2, 'poison': 1, 'ground': -2, 'rock': 1, 'bug': 2, 'ghost': 1, 'steel': -2, 'fire': 2, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 2, 'dragon': 1, 'dark': 1, 'fairy': 1},
            'bug': {'normal': 1, 'fighting': -2, 'flying': -2, 'poison': -2, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': -2, 'steel': -2, 'fire': -2, 'water': 1, 'grass': 2, 'electric': 1, 'psychic': 2, 'ice': 1, 'dragon': 1, 'dark': 2, 'fairy': -2},
            'ghost': {'normal': -4, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 2, 'steel': 1, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 2, 'ice': 1, 'dragon': 1, 'dark': -2, 'fairy': 1},
            'steel': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 2, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': -2, 'grass': 1, 'electric': -2, 'psychic': -2, 'ice': 2, 'dragon': 1, 'dark': 1, 'fairy': 2},
            'fire': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': -2, 'bug': 2, 'ghost': 1, 'steel': 2, 'fire': -2, 'water': -2, 'grass': 2, 'electric': 1, 'psychic': 1, 'ice': 2, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'water': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 2, 'rock': 2, 'bug': 1, 'ghost': 1, 'steel': 1, 'fire': 2, 'water': -2, 'grass': -2, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'grass': {'normal': 1, 'fighting': 1, 'flying': -2, 'poison': -2, 'ground': 2, 'rock': 2, 'bug': -2, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': 2, 'grass': -2, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'electric': {'normal': 1, 'fighting': 1, 'flying': 2, 'poison': 1, 'ground': -4, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': 1, 'fire': 1, 'water': 2, 'grass': -2, 'electric': -2, 'psychic': 1, 'ice': 1, 'dragon': -2, 'dark': 1, 'fairy': 1},
            'psychic': {'normal': 1, 'fighting': 2, 'flying': 1, 'poison': 2, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': -2, 'ice': 1, 'dragon': 1, 'dark': -4, 'fairy': 1},
            'ice': {'normal': 1, 'fighting': 1, 'flying': 2, 'poison': 1, 'ground': 2, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': -2, 'grass': 2, 'electric': 1, 'psychic': 1, 'ice': -2, 'dragon': 2, 'dark': 1, 'fairy': 1},
            'dragon': {'normal': 1, 'fighting': 1, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 2, 'dark': 1, 'fairy': -4},
            'dark': {'normal': 1, 'fighting': -2, 'flying': 1, 'poison': 1, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 2, 'steel': 1, 'fire': 1, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 2, 'ice': 1, 'dragon': 1, 'dark': -2, 'fairy': -2},
            'fairy': {'normal': 1, 'fighting': 2, 'flying': 1, 'poison': -2, 'ground': 1, 'rock': 1, 'bug': 1, 'ghost': 1, 'steel': -2, 'fire': -2, 'water': 1, 'grass': 1, 'electric': 1, 'psychic': 1, 'ice': 1, 'dragon': 2, 'dark': 2, 'fairy': 1}}

def calculate_multiplier(type, value):
    if type in boosted_types:
        return value * boost
    elif type in nerfed_types:
        return value * nerf
    return value

def get_defensive_score(type1, type2):
    types = [type1, int(type2)] if not type(type2) == np.str_ else [type1]
    score = 0
    for attacking_type, value in type_chart.items():
        local_scores = [0 if type(value[defending_type]) == float or abs(value[defending_type]) < 2 else value[defending_type] for defending_type in types]
        if 4 in local_scores or -4 in local_scores: score += 4
        elif not(sum(local_scores) == 0): score += -sum([calculate_multiplier(attacking_type, score) for score in local_scores])
    return score

def get_offensive_score(type1, type2):
    def eval_type(offensive_type):
        return np.sum([calculate_multiplier(defending_type, value) if type(value) != np.str_ and value != 0 else -4 for defending_type, value in type_chart[offensive_type].items()])
    return eval_type(type1) + eval_type(type2) if type(type2) != np.str_ else eval_type(type1)


def determine_fusion_types(types1, types2):
    types = []
    for i in range(len(types1)):
        type1_1, type1_2 = types1[i]; type2_1, type2_2 = types2[i]
        if type(type1_2) == float: type1_2 = type1_1
        if type(type2_2) == float: type2_2 = type2_1

        if type1_1 == type2_2: type2_2 = np.nan
        if type2_1 == type1_2: type1_2 = np.nan

        types.append([[type1_1, type2_2], [type2_1, type1_2]])
    return np.array(types)

Rating

In [70]:
def calc_fusion_stats(stats_pairs):
    stats1 = stats_pairs[:, 0, :]
    stats2 = stats_pairs[:, 1, :]
    fusion1 = np.array([stats1[:, 0] * 2/3 + stats2[:, 0] * 1/3,
                        stats1[:, 1] * 1/3 + stats2[:, 1] * 2/3,
                        stats1[:, 2] * 1/3 + stats2[:, 2] * 2/3,
                        stats1[:, 3] * 2/3 + stats2[:, 3] * 1/3,
                        stats1[:, 4] * 2/3 + stats2[:, 4] * 1/3,
                        stats1[:, 5] * 1/3 + stats2[:, 5] * 2/3]).T.astype(int)
    
    fusion2 = np.array([stats1[:, 0] * 1/3 + stats2[:, 0] * 2/3,
                        stats1[:, 1] * 2/3 + stats2[:, 1] * 1/3,
                        stats1[:, 2] * 2/3 + stats2[:, 2] * 1/3,
                        stats1[:, 3] * 1/3 + stats2[:, 3] * 2/3,
                        stats1[:, 4] * 1/3 + stats2[:, 4] * 2/3,
                        stats1[:, 5] * 2/3 + stats2[:, 5] * 1/3]).T.astype(int)
    return np.stack([fusion1, fusion2], axis=1)

def rate_pokemon(stats, multipliers, types, off_mul, def_mul):
    defensive_score = get_defensive_score(types[0], types[1]) / 20 * def_mul
    offensive_score = get_offensive_score(types[0], types[1]) / 10 * off_mul
    stat_score = (stats[0] * multipliers[0] + stats[1] * multipliers[1] + stats[2] * multipliers[2] + stats[3] * multipliers[3] + stats[4] * multipliers[4] + stats[5] * multipliers[5])  / 1000 * (1 - off_mul - def_mul)
    
    return (stat_score + defensive_score + offensive_score) * 30

def rate_fusion(stats, stat_multipliers, fusion_types, off_mul, def_mul):
    return [rate_pokemon(statn, stat_multipliers, fusion_typen, off_mul, def_mul) for statn, fusion_typen in zip(stats, fusion_types)]

### Main algorithm

A bit confusing, works with data in parallel.

In [71]:
def find_best_fusion(profile, dataset, cap, force_pokemon=None):
    pokemon = dataset[["Name", "Total", "Type 1", "Type 2", "HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed"]]
    pokemon = pokemon[pokemon["Total"] < cap]
    name_type_cols = ["Name", "Type 1", "Type 2"]
    stats_cols = ["HP", "Attack", "Defense", "Special Attack", "Special Defense", "Speed"]
    pairs = np.array(list(combinations(pokemon[name_type_cols + stats_cols].values, 2)))
    if force_pokemon:
        pairs = pairs[(pairs[:, 0, 0] == force_pokemon) | (pairs[:, 1, 0] == force_pokemon)]

    fusion_stats = calc_fusion_stats(pairs[:, :, 3:])
    fusion_types = determine_fusion_types(pairs[:, 0, 1:3], pairs[:, 1, 1:3])
    fusion_scores = rate_fusion(fusion_stats.reshape(-1, 6), profile[0], fusion_types.reshape(-1, 2), profile[1], profile[2])
    fusion_scores = np.array(fusion_scores).reshape(-1, 2)

    max_index = np.unravel_index(fusion_scores.argmax(), fusion_scores.shape)
    best_fusion_history = (pairs[max_index[0], 0, 0], pairs[max_index[0], 1, 0], fusion_scores[max_index], fusion_stats[max_index[0], max_index[1]])

    return [best_fusion_history]

### RESULTS

In [72]:
for profile_name, profile in profiles.items():
    print(profile_name)
    print(list(reversed(find_best_fusion(profile, pokemon_set, stat_cap, "Parasect")[-5:])))   # The last argument is optional 

Fast physical attacker
[('Parasect', 'Haxorus', 51.845699999999994, array([ 70, 112,  83,  60,  73,  52]))]
Fast special attacker
[('Parasect', 'Chandelure', 50.8485, array([ 60,  81,  83, 116,  86,  46]))]
Physical tank
[('Parasect', 'Steelix', 52.840799999999994, array([ 65,  88, 160,  58,  75,  30]))]
Special tank
[('Parasect', 'Snorlax', 53.15579999999999, array([126, 100,  75,  63, 100,  30]))]
Hybrid tank
[('Parasect', 'Aegislash', 50.42219999999999, array([ 60,  80, 103,  53, 126,  40]))]
Balanced overall
[('Parasect', 'Togekiss', 48.58200000000001, array([ 76,  80,  85, 100, 103,  46]))]
Physical wallbreaker
[('Parasect', 'Rhyperior', 52.67100000000001, array([ 96, 110,  96,  56,  63,  33]))]
Special wallbreaker
[('Parasect', 'Togekiss', 50.493, array([ 76,  80,  85, 100, 103,  46]))]
